In [ ]:
print("    Version control\n------------------------")
import os, fnmatch, random, math, sys, datetime
from pathlib import Path
import numpy as np;              print("Numpy\t\t", np.__version__)
import matplotlib as mpl;        print("matplotlib\t", mpl.__version__)
import matplotlib.pyplot as plt
import nibabel as nib;           print("NiBabel\t\t {}".format(nib.__version__))
from nibabel.testing import data_path
import pandas as pd;             print("Pandas\t\t {}".format(pd.__version__))
import imageio;                  print("imageio\t\t {}".format(imageio.__version__))
import h5py;                     print("H5py\t\t {}".format(h5py.__version__))
import sklearn;                  print("Scikit-learn\t {}".format(sklearn.__version__))
import skimage;                  print("Scikit-image\t {}".format(skimage.__version__))
import tensorflow as tf;         print("TensorFlow\t {}".format(tf.__version__))
import keras;                    print("Keras\t\t {}".format(keras.__version__))
import tensorflow.keras.backend as K
from tensorflow.keras import models, Input, Model
from tensorflow.keras.layers import Dense, Flatten, Reshape, BatchNormalization, Conv3D, MaxPooling3D, UpSampling3D, ZeroPadding3D
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.initializers import *
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.engine.topology import Layer, InputSpec
from keras.optimizers import SGD
from tensorflow.keras.utils import plot_model
from IPython.display import Image
from sklearn.cluster import KMeans
# %load_ext tensorboard       
# %reload_ext tensorboard

In [ ]:
## Load train data
data_source = "Data/data_random_1/" 
sample_train_subset = np.loadtxt(os.path.join(data_source, "sample_train.csv"), dtype=str, delimiter=",")
train_data = np.load(os.path.join(data_source, "train.npy")).reshape(100,182,218,182,1)
print('train_data shape is {}'.format(train_data.shape))

## Load validation data
sample_val_subset = np.loadtxt(os.path.join(data_source, "sample_valid.csv"), dtype=str, delimiter=",")
valid_data = np.load(os.path.join(data_source, "valid.npy")).reshape(24,182,218,182,1)
print('valid_data shape is {}'.format(valid_data.shape))

In [ ]:
## Padding datasets
train_pad_data = (ZeroPadding3D(padding=(5,19,5))(train_data)).numpy()
print('train_padded_data shape is {}'.format(train_pad_data.shape))

valid_pad_data = (ZeroPadding3D(padding=(5,19,5))(valid_data)).numpy()
print('valid_padded_data shape is {}'.format(valid_pad_data.shape))

In [ ]:
## Convolutional AutoEncoder Model Designe:

IMAGE_HEIGHT = train_pad_data.shape[1]
IMAGE_WIDTH = train_pad_data.shape[2]
IMAGE_DEPTH = train_pad_data.shape[3]
BATCH_SIZE = 1
EPOCHS = 1
data_shape = [1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, 1]
input_shape = [BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, 1]
print("Padded data shape: {}\n".format(input_shape))

## Encoder
input_img = Input(shape=(192, 256, 192, 1), name='Input')
x = Conv3D(filters=128, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv1')(input_img)
x = BatchNormalization(name='BN_Conv1')(x)
x = Conv3D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv2')(x)
x = BatchNormalization(name='BN_Conv2')(x)
x = Conv3D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv3')(x)
x = BatchNormalization(name='BN_Conv3')(x)
x = Conv3D(filters=16, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv4')(x)
x = BatchNormalization(name='BN_Conv4')(x)
x = Conv3D(filters=8, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv5')(x)
x = BatchNormalization(name='BN_Conv5')(x)

## Latent Features
shape_before_flattening = tf.keras.backend.int_shape(x)
x_LF = Flatten(name='LF')(x)

##____________________________________________________________________________________________________
# init = VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform')
# encoded = Dense(50, kernel_initializer=init, activation='relu', name='encoded')(x_LF)
# x_LF = Dense(100, activation='relu', name='encoded')(x_LF)
##____________________________________________________________________________________________________

encoded = x_LF    # Hadi for L200

##____________________________________________________________________________________________________
# x = BatchNormalization(name='BN_Dense')(encoded)
# # x = Dense(np.prod(shape_before_flattening[1:]), activation='relu', kernel_initializer=init)(encoded)
# x = Dense(np.prod(shape_before_flattening[1:]), activation='relu')(x)
# encoded = x
##____________________________________________________________________________________________________

x = Reshape(shape_before_flattening[1:], name='UnFlat')(encoded)

## Decoder
x = Conv3D(filters=8, kernel_size=3, padding='same', activation='relu', name='DeConv2')(x)
x = BatchNormalization(name='BN_DeConv2')(x)
x = UpSampling3D(size=(2, 2, 2), name='UpSampling2')(x)
x = Conv3D(filters=16, kernel_size=3, padding='same', activation='relu', name='DeConv3')(x)
x = BatchNormalization(name='BN_DeConv3')(x)
x = UpSampling3D(size=(2, 2, 2), name='UpSampling3')(x)
x = Conv3D(filters=32, kernel_size=3, padding='same', activation='relu', name='DeConv4')(x)
x = BatchNormalization(name='BN_DeConv4')(x)
x = UpSampling3D(size=(2, 2, 2), name='UpSampling4')(x)
x = Conv3D(filters=64, kernel_size=3, padding='same', activation='relu', name='DeConv5')(x)
x = BatchNormalization(name='BN_DeConv5')(x)
x = UpSampling3D(size=(2, 2, 2), name='UpSampling5')(x)
x = Conv3D(filters=128, kernel_size=3, padding='same', activation='relu', name='DeConv6')(x)
x = BatchNormalization(name='BN_DeConv6')(x)
x = UpSampling3D(size=(2, 2, 2), name='UpSampling6')(x)
decoded = Conv3D(filters=1, kernel_size=3, padding='same', activation='sigmoid', name='Output')(x)

model_CAE = Model(inputs=input_img, outputs=decoded, name='AutoEncoder')
## optimizer=rmsprop, sgd
model_CAE.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model_CAE.summary()
# plot_model(model_CAE, to_file='Convolutional_autoencoder_model.png', show_shapes=True)
# Image(filename='Convolutional_autoencoder_model.png')

In [ ]:
## Start time:
from datetime import datetime
start_time = datetime.now().strftime("%Y_%m_%d___%H_%M"); print("\nStart Time =", start_time, "\n")

## Model Fit
# model_CAE.load_weights(os.path.join("Weights/L1.hdf5"), by_name=True)  # 1
# model_CAE.load_weights(os.path.join("Weights/L200___2021_01_03___05_48.hdf5"))  # 100
# model_CAE.load_weights(os.path.join("Weights/L100___2021_01_01___23_57.hdf5"), by_name=True)  # 500
# model_CAE.load_weights(os.path.join("Weights/L100___2021_01_02___08_37.hdf5"), by_name=True)  # 1000

model_checkpoint_callback = ModelCheckpoint(filepath=os.path.join("Check/P5_2304___" + start_time), save_weights_only=True, save_best_only=True, monitor='val_loss', mode='max', verbose=1) 
tb_callback = TensorBoard(os.path.join("Logs/P5_2304___" + start_time), histogram_freq=1)
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=5, mode='auto')
# model_CAE.fit(train_data[0:10,:], train_data[0:10,:], validation_data=(valid_data, valid_data), epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, callbacks=[tb_callback, model_checkpoint_callback], verbose=1)
model_CAE.fit(train_data, train_data, validation_data=(valid_data, valid_data), epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, callbacks=[tb_callback, model_checkpoint_callback], verbose=1)
model_CAE.save_weights(os.path.join("Weights/P5_2304___" + start_time + ".hdf5"))

## End time:
from datetime import datetime
end_time = datetime.now().strftime("%Y_%m_%d___%H_%M"); print("\nEnd Time =", end_time)

In [ ]:
## Reconstructing data 

test_pad_data = train_pad_data[0,:].reshape(1, 192, 256, 192, 1)
reconstructed = model_CAE.predict(test_pad_data)

# for m in range(1):
#     slice_0 = test_pad_data[m, 96, :, :, 0]
#     slice_1 = test_pad_data[m, :, 128, :, 0]
#     slice_2 = test_pad_data[m, :, :, 96, 0]
#     show_slices([slice_0, slice_1, slice_2])
#     plt.suptitle(sample_train_subset[m], x=0.5, y=0.9)

In [ ]:
print('\ntrain_pad_data[0,100,100:105,100]\n\n {}'.format(train_pad_data[0,100,100:105,100]),'\n')
print('\nReconstructed_data[0,100,100:105,100]\n\n {}'.format(reconstructed[0,100,100:105,100]),'\n')

In [ ]:
model_Encoder = Model(inputs=input_img, outputs=encoded, name='Encoder')
model_Encoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
# model_Encoder.summary()
# plot_model(model_Encoder, to_file='Encoder_model.png', show_shapes=True)
# Image(filename='Encoder_model.png')

In [ ]:
test_pad_data = train_pad_data[0,:].reshape(1, 192, 256, 192, 1)
LF_features = model_Encoder.predict(test_pad_data)[0]

print('LF_features size: {} -------- {} \n'.format(LF_features.shape, type(LF_features)))
print('The first feature is {}\n'.format(LF_features[0]))
print(LF_features)